In [ ]:
import importlib
from mkids import TopSoc
import numpy as np
import matplotlib.pyplot as plt
import Scan
from tqdm.notebook import trange
soc = TopSoc()

In [ ]:
importlib.reload(Scan)
scan = Scan.Scan(soc)

In [ ]:
#fMixer = soc.fsOut/2
fMixer=4500
soc.set_mixer(fMixer)

In [ ]:
# One frequency per output bin, offset a little bit from the middle
outChs = np.arange(soc.nOutCh)
freqs = np.sort(soc.outCh2FreqCenter(outChs))
inChs, inOffsets = soc.inFreq2chOffset(freqs)
#for i in range(soc.nOutCh):
#    print("freq=%8.3f  outCh=%3d  inCh=%4d   inOffset=%+5.3f"%(freqs[i],outChs[i], inChs[i], inOffsets[i]))
    #print(i, outChs[i], freqs[i])

In [ ]:
nFreqs = len(freqs)
amps = np.array([0.9])
fis = np.array([0])
allxs = np.zeros(nFreqs, dtype=complex)
for i in trange(nFreqs):
    fTone = freqs[i]
    scan.setTones(np.array([fTone]), amps, fis)
    scan.prepRead(decimation=2)
    xs = scan.readAndUnpack(nsamp=10000, iBegin=1000, average=True)
    allxs[i] = xs[0]

In [ ]:
plt.plot(freqs, 10*np.log10(np.abs(allxs)))
#plt.plot(freqs, np.abs(allxs))
plt.xlabel("frequency (MHz)")
plt.ylabel("amplitude measured dB")
plt.title("board=%s fMixer=%.1f"%(soc.board, soc.get_mixer()))
plt.savefig("temp.png")


In [ ]:
freqs.shape

In [ ]:
plt.plot(freqs, np.angle(allxs), ',')
#plt.plot(freqs, np.abs(allxs))
plt.xlabel("frequency (MHz)")
plt.ylabel("phi")
plt.title("board=%s fMixer=%.1f"%(soc.board, soc.get_mixer()))


In [ ]:
soc.dacs

In [ ]:
fn = soc.fsIn/2
print("fn =",fn)
nZone = 2
fTone = (nZone-.5)*fn + 20
print("fTone =",fTone)
fMixer = fTone - 250
soc.set_mixer(fMixer)
print(soc.inFreq2chOffset(fTone))
freqs = np.array([fTone])
amps = np.array([0.9])
fis = np.array([0.0])
bandwidth = soc.fcOut/100
nf = 10
decimation = 2
nt = 1
fscan = scan.fscan(freqs, amps, fis, bandwidth, nf, decimation, nt, doProgress=True)

In [ ]:
Scan.fscanPlot(fscan,0)

In [ ]:
fAliased = soc.fAliasedFromFTone(freqs)
print(fAliased)

In [ ]:
outCh = soc.outFreq2ch(freqs)
print(outCh)

In [ ]:
importlib.reload(Scan)
scan = Scan.Scan(soc)
nominalDelay = scan.measureNominalDelay(outCh[0], decimation=2, doPlot=True)